In [1]:
# Drive importieren
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import imageio
import pandas as pd
from torch.optim.lr_scheduler import StepLR

from torchsummary import summary

In [4]:
# für die Ausführung auf Colab, sonst wird auf CPU trainiert 
if torch.cuda.is_available():
   device = torch.device('cuda:0')
   print('running on gpu')
else:
   device = torch.device('cpu')
   print('running on cpu')

running on cpu


In [5]:
# initialisiere random seed (reproduzierbar)
torch.manual_seed(0)   

In [7]:
# Lade Daten (gilt nur für colab)
# data_raw= pd.read_csv('/content/drive/My Drive/gasflowdata_mitA_16dim.csv')

# Lade Daten von lokaler Festplatte
data_raw= pd.read_csv('gasflowdata_mitA_16dim.csv')
data_raw.head()

########################################
# Gib mir max und min aus
# print(np.max(data_raw['dilution']))
# print(np.max(data_raw['lipids']))
# print(np.max(data_raw['carbohydrates']))
# print(np.max(data_raw['proteins']))
# print(np.min(data_raw['dilution']))
# print(np.min(data_raw['lipids']))
# print(np.min(data_raw['carbohydrates']))
# print(np.min(data_raw['proteins']))

# print(np.max(data_raw['Gasflow']))
# print(np.min(data_raw['Gasflow']))

# data_raw['time_1'][0]
########################################

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,...,time_11,time_12,time_13,time_14,time_15,time_16,carbohydrates,proteins,lipids,dilution
0,1691.696473,0.194330,0.027472,0.020199,0.019492,0.019414,0.019405,0.019404,0.019404,0.019404,...,148.484848,178.787879,209.090909,239.393939,269.69697,300,0.814724,0.079125,0.106152,0.133840
1,949.997750,0.038927,0.013907,0.013027,0.012980,0.012977,0.012977,0.012977,0.012977,0.012977,...,148.484848,178.787879,209.090909,239.393939,269.69697,300,0.905792,0.090004,0.004204,0.179608
2,2507.140873,1.407618,0.086091,0.034676,0.027808,0.026578,0.026338,0.026290,0.026280,0.026278,...,148.484848,178.787879,209.090909,239.393939,269.69697,300,0.126987,0.632277,0.240736,0.095542
3,1002.798536,0.045099,0.014782,0.013675,0.013611,0.013607,0.013607,0.013607,0.013607,0.013607,...,148.484848,178.787879,209.090909,239.393939,269.69697,300,0.913376,0.050319,0.036305,0.174718
4,1316.125423,0.090699,0.019329,0.016368,0.016138,0.016119,0.016117,0.016117,0.016117,0.016117,...,148.484848,178.787879,209.090909,239.393939,269.69697,300,0.632359,0.198621,0.169020,0.152592


In [8]:
# # max-standardize dilution
# gas = data_raw.drop(['Gasflow_start', 'Gasflow_6'], axis=1)
# # gas = data_raw

# for field in ['dilution','Gasflow_1', 'Gasflow_2', 'Gasflow_3', 'Gasflow_4', 
#               'Gasflow_5', 'Gasflow_end']:
# # for field in ['dilution', 'Gasflow_end']:
#     max_field = gas[field].max()
#     gas.loc[:,field] = (gas[field])/max_field
#     # min_field = gas[field].min()
#     # gas.loc[:,field] = (gas[field] - min_field)/(max_field - min_field)
    
#     print(gas[field].min())
#     print(gas[field].max())
    
# gas.head()

In [9]:
# 'gasflow_' + str(6)

In [10]:
# max-standardize dilution
gas = data_raw
for ii in range(16):
    gas = gas.drop(['time_'+str(ii+1)], axis=1)
    print(ii)
    
gas.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,1691.696473,0.194330,0.027472,0.020199,0.019492,0.019414,0.019405,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.814724,0.079125,0.106152,0.133840
1,949.997750,0.038927,0.013907,0.013027,0.012980,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.905792,0.090004,0.004204,0.179608
2,2507.140873,1.407618,0.086091,0.034676,0.027808,0.026578,0.026338,0.026290,0.026280,0.026278,0.026277,0.026277,0.026277,0.026277,0.026277,0.026277,0.126987,0.632277,0.240736,0.095542
3,1002.798536,0.045099,0.014782,0.013675,0.013611,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.913376,0.050319,0.036305,0.174718
4,1316.125423,0.090699,0.019329,0.016368,0.016138,0.016119,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.632359,0.198621,0.169020,0.152592


In [11]:
# for field in ['dilution']:
# # for field in ['dilution', 'Gasflow_end']:
#     max_field = gas[field].max()
#     gas.loc[:,field] = (gas[field])/max_field

In [12]:
# Erstelle String-Liste 
gasflow_list = []
for ii in range(16):
    gasflow_list.append('Gasflow_' + str(ii+1))

print(gasflow_list)

['Gasflow_1', 'Gasflow_2', 'Gasflow_3', 'Gasflow_4', 'Gasflow_5', 'Gasflow_6', 'Gasflow_7', 'Gasflow_8', 'Gasflow_9', 'Gasflow_10', 'Gasflow_11', 'Gasflow_12', 'Gasflow_13', 'Gasflow_14', 'Gasflow_15', 'Gasflow_16']


In [13]:
# normalisiere Gasfluss
for field in [gasflow_list]:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

gas.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,0.645230,0.000069,0.000010,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.814724,0.079125,0.106152,0.133840
1,0.362338,0.000014,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.905792,0.090004,0.004204,0.179608
2,0.956248,0.000497,0.000030,0.000014,0.000011,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.126987,0.632277,0.240736,0.095542
3,0.382477,0.000016,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.913376,0.050319,0.036305,0.174718
4,0.501983,0.000032,0.000007,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.632359,0.198621,0.169020,0.152592


In [15]:
# teile in 90% Trainingsdaten und 10% Testdaten
sample = np.random.choice(gas.index, size=int(len(gas)*0.9), replace=False)
data_raw, test_data = gas.loc[sample], gas.drop(sample)

# teile in features und targets
# columns = ['Gasflow_1',...,'Gasflow_end']

columns = gasflow_list

features, targets = data_raw.drop(columns, axis=1), data_raw[columns]
features_test, targets_test = test_data.drop(columns, axis=1), test_data[columns]

In [16]:
features_test.head()

,carbohydrates,proteins,lipids,dilution
6,0.278498,0.003628,0.717873,0.125264
17,0.915736,0.022596,0.061668,0.145004
18,0.792207,0.158721,0.049072,0.141163
19,0.959492,0.032629,0.007878,0.147949
20,0.655741,0.035890,0.308369,0.093797


In [17]:
targets_test.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16
6,0.715345,0.000110,0.000013,0.000010,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
17,0.558951,0.000044,0.000008,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
18,0.585579,0.000049,0.000008,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
19,0.536914,0.000039,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
20,0.977057,0.000623,0.000037,0.000016,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012


In [18]:
# Überführe alle Daten in Tensoren
x_data = torch.tensor(features.values, dtype=torch.float)
y_data = torch.tensor(targets.values, dtype=torch.float)

x_test = torch.tensor(features_test.values, dtype=torch.float)
y_test = torch.tensor(targets_test.values, dtype=torch.float)

In [19]:
print(y_data, x_data)

tensor([[1.3340e-01, 2.7372e-01, 2.6973e-01,  ..., 3.0026e-01, 3.0027e-01,
         3.0027e-01],
        [4.5887e-01, 2.5469e-05, 6.2410e-06,  ..., 5.9935e-06, 5.9935e-06,
         5.9935e-06],
        [3.8791e-01, 1.7474e-05, 5.5976e-06,  ..., 5.6856e-06, 5.6856e-06,
         5.6856e-06],
        ...,
        [1.7210e-01, 4.2488e-01, 4.2671e-01,  ..., 4.7686e-01, 4.7686e-01,
         4.7686e-01],
        [7.7719e-01, 1.4646e-04, 1.4091e-05,  ..., 8.4867e-06, 8.4867e-06,
         8.4867e-06],
        [2.8238e-01, 7.4982e-01, 7.5138e-01,  ..., 8.3526e-01, 8.3526e-01,
         8.3526e-01]])


In [28]:
# Erstelle neuronales Feedforward-Netz:
p = 0.2 # für den Fall, dass Dropout inkludiert wird

net = torch.nn.Sequential(
        torch.nn.Linear(4, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 16),
        torch.nn.ReLU(),
        torch.nn.Linear(16, 16),
    ).to(device)

# gibt nur Parameteranzahl zurück
# print(sum([param.nelement() for param in net.parameters()]))

# ggf. ändern, je nachdem wo das Netz hinterlegt ist
# summary(net.cuda(), (2,4)) # Fasse das Netz zusammen
summary(net.cpu(), (2,4)) # Fasse das ganze Netz zusammen

510528
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 2, 512]           2,560
              ReLU-2               [-1, 2, 512]               0
            Linear-3               [-1, 2, 512]         262,656
              ReLU-4               [-1, 2, 512]               0
            Linear-5               [-1, 2, 256]         131,328
              ReLU-6               [-1, 2, 256]               0
            Linear-7               [-1, 2, 256]          65,792
              ReLU-8               [-1, 2, 256]               0
            Linear-9               [-1, 2, 128]          32,896
             ReLU-10               [-1, 2, 128]               0
           Linear-11                [-1, 2, 64]           8,256
             ReLU-12                [-1, 2, 64]               0
           Linear-13                [-1, 2, 64]           4,160
             ReLU-14            

In [29]:
# MSE Loss Funktion
def my_loss(output, target):
    loss = (0.5*(output - target)**2).sum()
    return loss

In [30]:
# definiere Art der Optimierung
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9, dampening=0.2)

# Für den Fall dass torch eigene Optimierungsfunktion verwendet werden soll
# loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

# Definiere Scheduler (falls normale Optimierung nicht funktioniert)
# gamma = decaying factor
# scheduler = StepLR(optimizer, step_size=1000, gamma=0.1)

In [31]:
# Optimierungsparameter
BATCH_SIZE = 256
EPOCH = 10000

In [32]:
# Welche Daten sollen zum Training genutzt werden?
torch_dataset = Data.TensorDataset(x_data, y_data)
print(torch_dataset)

In [33]:
# Definiere Loader-Objekt 
loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True)
print(loader)

In [ ]:
last_loss = None
# start training
for epoch in range(EPOCH):
    for step, (x_data, y_data) in enumerate(loader): # for each training step
        # schaufel Daten auf Device (CPU oder GPU)
        x_data_1 = x_data.to(device)
        y_data_1 = y_data.to(device)
        prediction = net(x_data_1)     # input x and predict based on x

        loss = my_loss(prediction, y_data_1)     # must be (1. nn output, 2. target)
        
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients
    if epoch % 100 == 0:
        if last_loss and last_loss < loss:
            print('Epoch: ',epoch, 'Train loss: ', loss, '  WARNING - Loss Increasing')
        else:
            print('Epoch: ',epoch, 'Train loss: ', loss)
        last_loss = loss

# Print model's state_dict
model = net
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

# gilt für colab und Googledrive
# path = '/content/drive/My Drive/checkp_xy_mit_16dim_2.pth.tar'

# gilt für lokales Laufwerk
path = 'checkp_xy_mit_16dim_2.pth.tar'
checkpoint = {'x_test': x_test, 'y_test': y_test, 'x_data': x_data, 'y_data': y_data,
              'model': model,
              'state_dict': model.state_dict()}

torch.save(checkpoint, path)

# Training zu Ende